In [2]:
import os
import sys
import time
import glob
import numpy as np
import logging
import argparse
import tensorflow as tf
import numpy as np
import utils
from tqdm import tqdm
import shutil
from model_search import Network
from architect_graph import Architect

import time
from genotypes import PRIMITIVES
from genotypes import Genotype
from scipy.special import softmax
import utils

In [3]:
# tf.enable_eager_execution()
tf.set_random_seed(6969)
np.random.seed(6969)

In [4]:
args = {
    "momentum": 0.9,
    "weight_decay": 3e-4,
    "arch_learning_rate": 3e-1,
    "momentum": 0.9,
    "grad_clip": 5,
    "learning_rate": 0.9,
    "learning_rate_decay": 0.97,
    "learning_rate_min": 0.0001,
    "num_batches_per_epoch": 2000,
    
    "unrolled": True,
    "epochs": 10,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    "save": "EXP",
    "init_channels": 3,
    "num_layers": 3,
    "num_classes": 6,
    "crop_size": [8, 8],
    "save_checkpoints_steps": 100,
    "model_dir": 'gs://unet-darts/train-search-ckptss',
    "max_steps": 10000,
    "num_shards": 8,
    # NEW
    "steps_per_eval": 2,
    "num_train_examples": 16,
    "num_batches_per_epoch": 2,
    "iterations_per_loop": 50,
    #
    "seed": 6969,
    
    "use_tpu": False,
    "use_host_call": True,
    "tpu": 'unet-darts',
    "zone": 'us-central1-f',
    "project": "isro-nas",
    "use_bfloat": False
}

class Struct:
    def __init__(self, **entries):
        self.__dict__.update(entries)

args = Struct(**args)

### input_fn

In [5]:
def make_inp_fn(filename, mode, batch_size):
    
    def _input_fn(params):
        image_dataset = tf.data.TFRecordDataset(filename)
        W, H = args.crop_size[0], args.crop_size[1]

        # Create a dictionary describing the features.  
        image_feature_description = {
            'train_name': tf.FixedLenFeature([], tf.string),  
            'train_x': tf.FixedLenFeature([], tf.string),
            'train_y': tf.FixedLenFeature([], tf.string),
            'valid_name': tf.FixedLenFeature([], tf.string),  
            'valid_x': tf.FixedLenFeature([], tf.string),
            'valid_y': tf.FixedLenFeature([], tf.string)
        }
        def _parse_image_function(example_proto):
            # Parse the input tf.Example proto using the dictionary above.
            feature= tf.parse_single_example(example_proto, image_feature_description)
            train_x = feature['train_x']
            train_y = feature['train_y']
            valid_x = feature['valid_x']
            valid_y = feature['valid_y']
            train_name = feature['train_name']
            valid_name = feature['valid_name']

            train_x = tf.image.decode_png(train_x, channels=3)
            train_y = tf.image.decode_png(train_y, channels=3)
            valid_x = tf.image.decode_png(valid_x, channels=3)
            valid_y = tf.image.decode_png(valid_y, channels=3)
            
            train_x = tf.image.resize(train_x, (W, H))
            train_y = tf.image.resize(train_y, (W, H))
            valid_x = tf.image.resize(valid_x, (W, H))
            valid_y = tf.image.resize(valid_y, (W, H))
            
            train_y = train_y[:, :, 0]
            train_y = tf.expand_dims(train_y, axis=-1)
            
            valid_y = valid_y[:, :, 0]
            valid_y = tf.expand_dims(valid_y, axis=-1)
            
            if(args.use_bfloat):
                train_x = tf.cast(train_x, tf.bfloat16)
                train_y = tf.cast(train_y, tf.bfloat16)
                valid_x = tf.cast(valid_x, tf.bfloat16)
                valid_y = tf.cast(valid_y, tf.bfloat16)
            else:
                train_x = tf.cast(train_x, tf.float32)
                train_y = tf.cast(train_y, tf.float32)
                valid_x = tf.cast(valid_x, tf.float32)
                valid_y = tf.cast(valid_y, tf.float32)
                
            return ((train_x, valid_x), (train_y, valid_y))

        dataset = image_dataset.map(_parse_image_function)
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = None # indefinitely
            dataset = dataset.shuffle(buffer_size = 10 * batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs).batch(batch_size,  drop_remainder=True)

        return dataset
    
    return _input_fn

In [6]:
def make_inp_fn2(filename, mode, batch_size):
    
    def _input_fn(params):
        W, H = args.crop_size[0], args.crop_size[1]
        NUM_IMAGES = 20
        x_train = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_train = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)
        x_valid = np.random.randint(0, 256, (NUM_IMAGES, W, H, 3)).astype(np.float32)
        y_valid = np.random.randint(0, args.num_classes, (NUM_IMAGES, W, H, 1)).astype(np.float32)
        
        if(args.use_bfloat):
            x_train, x_valid = tf.convert_to_tensor(x_train, dtype=tf.bfloat16), tf.convert_to_tensor(x_train, dtype=tf.bfloat16)
            y_train, y_valid = tf.convert_to_tensor(y_train, dtype=tf.bfloat16), tf.convert_to_tensor(y_valid, dtype=tf.bfloat16)
        ds = (x_train, x_valid), (y_train, y_valid)
        
        dataset = tf.data.Dataset.from_tensor_slices(ds)
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)

        dataset = dataset.repeat(num_epochs).batch(batch_size, drop_remainder=True)
        return dataset
    
    return _input_fn

### Model function

In [7]:
class GeneSaver(tf.estimator.SessionRunHook):
    def __init__(self, model, alphas_normal, alphas_reduce):
        self.model = model
        self.alphas_normal = alphas_normal
        self.alphas_reduce = alphas_reduce
    
    def begin(self):
        self.global_step = tf.train.get_or_create_global_step()
        
    def end(self, session):
        alphas_normal, alphas_reduce = session.run([self.alphas_normal, self.alphas_reduce])
        alphas_normal = softmax(alphas_normal)
        alphas_reduce = softmax(alphas_reduce)
        
        genotype = self.model.genotype(alphas_normal, alphas_reduce)        
        self.global_step = session.run(self.global_step)
        
        filename = 'final_genotype.{}'.format((self.global_step))
        tf.logging.info("Saving Genotype for step: {}".format(str(self.global_step)))
        utils.write_genotype(genotype, filename)

In [8]:
def model_fn(features, labels, mode, params):
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes)
    
    eval_hooks = None
    loss = None
    train_op = None
    eval_metric_ops = None
    prediction_dict = None
    export_outputs = None
    host_call = None
    # 2. Loss function, training/eval ops
    if mode == tf.estimator.ModeKeys.TRAIN:
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        global_step = tf.train.get_global_step()
        learning_rate_min = tf.constant(args.learning_rate_min)

        learning_rate = tf.train.exponential_decay(
            args.learning_rate,
            global_step,
            decay_rate=args.learning_rate_decay,
            decay_steps=args.num_batches_per_epoch,
            staircase=False,
        )
        lr = tf.maximum(learning_rate, learning_rate_min)
        optimizer = tf.train.MomentumOptimizer(lr, args.momentum)

        if(args.use_tpu):
            optimizer = tf.tpu.CrossShardOptimizer(optimizer)


        preds = model(x_train)
        architect = Architect(model, args)
        # architect step
        tf.logging.info("Computing Architect step!")
        architect_step = architect.step(input_train=x_train,
                                        target_train=y_train,
                                        input_valid=x_valid,
                                        target_valid=y_valid,
                                        unrolled=args.unrolled,
                                        )


        w_var = model.get_thetas()
        loss = model._loss(preds, y_train)

        with tf.control_dependencies([architect_step]):
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                train_op = optimizer.minimize(loss, var_list=w_var, global_step=global_step)
        
        # y = one_hot_encode(y_train)
        
        if args.use_host_call:
            def host_call_fn(global_step, learning_rate, loss):
                # Outfeed supports int32 but global_step is expected to be int64.
                global_step = tf.reduce_mean(global_step)
                global_step = tf.cast(global_step, tf.int64)
                # acc = tf.metrics.accuracy(labels=y, 
                #                             predictions=one_hot_encode(tf.expand_dims(tf.argmax(preds, axis=-1))))
                
                with (tf.contrib.summary.create_file_writer(args.model_dir).as_default()):
                    with tf.contrib.summary.always_record_summaries():
                        tf.contrib.summary.scalar(
                            'learning_rate', tf.reduce_mean(learning_rate),
                            step=global_step)
                        tf.contrib.summary.scalar(
                            'loss', tf.reduce_mean(loss),step=global_step)
                return tf.contrib.summary.all_summary_ops()

            global_step_t = tf.reshape(global_step, [1])
            learning_rate_t = tf.reshape(learning_rate, [1])
            loss_t = tf.reshape(loss, [1])
            host_call = (host_call_fn,
                           [global_step_t, learning_rate_t, loss_t])

    elif mode == tf.estimator.ModeKeys.EVAL:
        alphas_normal = model.arch_parameters()[0]
        alphas_reduce = model.arch_parameters()[1]
        gene_saver = GeneSaver(model, alphas_normal, alphas_reduce)
        eval_hooks = [gene_saver]
        
        (x_train, x_valid) = features
        (y_train, y_valid) = labels
        preds = model(x_valid)
        loss = model._loss(preds, y_valid)
        y = one_hot_encode(y_valid)
        print(y)
        
        metric_fn = lambda y, preds: {
            'miou': tf.metrics.mean_iou(
                    labels=y,
                    predictions=one_hot_encode(tf.expand_dims(tf.argmax(preds, axis=-1), axis=-1)),
                    num_classes=args.num_classes
                ),
        
            'acc': tf.metrics.accuracy(labels=y, 
                                      predictions=one_hot_encode(tf.expand_dims(tf.argmax(preds, axis=-1), axis=-1)))
        }
        eval_metric_ops = (metric_fn, [y, preds])
        
    elif mode == tf.estimator.ModeKeys.PREDICT:
        x = features
        y = labels
        preds = model(x)
        prediction_dict = {"predictions": preds}
        export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = preds)}
    
    # 5. Return EstimatorSpec
    return tf.estimator.tpu.TPUEstimatorSpec(
        mode = mode,
        predictions = prediction_dict,
        loss = loss,
        train_op = train_op,
        eval_metrics = eval_metric_ops,
        export_outputs = export_outputs,
        evaluation_hooks=eval_hooks,
        host_call=host_call
    )

### Estimator

In [9]:
# Create functions to read in respective datasets
def get_train():
    return make_inp_fn(filename = tf.gfile.Glob('gs://unet-darts/datasets/train-search/train-search-*-00008.tfrecords'),
                        mode = tf.estimator.ModeKeys.TRAIN,
                        batch_size = args.train_batch_size)

In [10]:
# Create serving input function
def serving_input_fn():
    feature_placeholders = {
      IMAGE_LOC: tf.placeholder(tf.float32, [None])
    }
    
    feature_placeholders['IMAGES'] = tf.placeholder(tf.float32, [None, args.crop_size[0], args.crop_size[1], args.init_channels])
    
    features = {
    key: tf.expand_dims(tensor, -1)
    for key, tensor in feature_placeholders.items()
    }

    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders)

In [11]:
tpu_cluster_resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
      tpu=args.tpu,
      zone=args.zone,
      project=args.project)

In [12]:
config = tf.estimator.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=args.model_dir,
    save_checkpoints_steps=args.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
      num_shards=args.num_shards,
      iterations_per_loop=args.iterations_per_loop),
    tf_random_seed=int(args.seed)
)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [13]:
estimator = tf.estimator.tpu.TPUEstimator(
    use_tpu=args.use_tpu,
    model_fn=model_fn,
    config=config,
    train_batch_size=args.train_batch_size,
    eval_batch_size=args.eval_batch_size,
    params=vars(args)
)

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.240.1.18:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd7fc4e8ed0>, '_model_dir': 'gs://unet-darts/train-search-ckptss', '_protocol': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tpu_config': TPUConfig(iterations_per_loop=50, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None, eval_training_input_configuration=2, experimental_host_call_every_n_steps=1), '_tf_random_seed': 6969, '_save_summary_steps': 100, '_device_fn': None, '_session_creation_tim

In [14]:
estimator.train(input_fn = get_train(), steps=1)
# Loss for final step: 1.9186516.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU




INFO:tensorflow:Computing Architect step!



[None, None]
ERROR:tensorflow:Error recorded from training_loop: unsupported operand type(s) for -: 'NoneType' and 'NoneType'
INFO:tensorflow:training_loop marked as finished


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

In [32]:
estimator.get_variable_value('network/alphas_normal')
# array([[2.6859643e-05, 9.8239048e-04],

array([[ 0.595209  , -0.7686217 ,  0.1269211 ],
       [ 1.2717432 , -1.2979747 ,  1.2952019 ],
       [ 0.27162904, -0.7305325 ,  1.0984756 ],
       [ 0.6433977 , -0.20175919, -0.66851974],
       [-0.64537674,  0.5257004 , -0.22668937],
       [ 1.4006627 ,  0.5730785 , -1.0582085 ],
       [-0.45131016,  1.2660947 , -0.63371235],
       [ 0.7567219 , -0.6705638 ,  0.5534028 ],
       [ 1.3372443 ,  1.1384563 , -1.2391193 ],
       [ 0.73247147, -1.2732375 ,  0.06622033],
       [-0.39793304,  0.6117226 , -1.0894794 ],
       [ 1.4384155 , -1.0603315 ,  0.5252124 ],
       [ 0.11675408, -1.0610735 ,  1.2649337 ],
       [ 0.8676318 ,  0.1294182 , -1.5662566 ]], dtype=float32)

In [33]:
estimator.get_variable_value('network/sequential/conv2d/kernel:0')
# array([[[[ 0.21839981, -0.21216999,  0.21580444,  0.13639389,

array([[[[ 2.05207974e-01, -2.54415035e-01,  1.78845093e-01,
           5.62108904e-02,  1.75728090e-02,  1.36387229e-01,
          -2.52650678e-02,  3.62207778e-02,  1.87400669e-01],
         [-1.17236957e-01, -2.43756577e-01,  1.49820089e-01,
          -1.64522156e-01, -2.50869781e-01,  6.89824224e-02,
          -1.08388692e-01, -1.87773913e-01,  7.43458420e-03],
         [-1.25536025e-01,  1.27577946e-01, -1.02268748e-01,
          -8.75649601e-02,  4.22250181e-02,  7.88645446e-02,
          -5.79144880e-02,  2.02150822e-01, -1.24112681e-01]],

        [[-2.02646643e-01,  1.91910967e-01, -1.43239602e-01,
           9.40711424e-03, -1.79918706e-01, -1.87509865e-01,
          -1.23070486e-01, -1.52189299e-01, -1.09877579e-01],
         [ 1.11750767e-01, -1.28562778e-01,  2.14938708e-02,
           1.52220428e-02, -2.44464185e-02, -3.01226974e-04,
           5.14815897e-02, -1.88301682e-01,  2.43121758e-02],
         [-1.27230629e-01,  1.95639133e-01,  1.32539049e-02,
          -1.2022

## Extra Test

In [15]:
with tf.tpu.bfloat16_scope():
    criterion = tf.losses.softmax_cross_entropy
    model = Network(C=args.init_channels, net_layers=args.num_layers, criterion=criterion, num_classes=args.num_classes, scope='shit')

In [16]:
(x_train, x_valid), (y_train, y_valid) = get_train()(vars(args)).make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [17]:
logits = model(x_train)

In [18]:
loss = model._loss(logits, y_train)

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.001)
optimizer.compute_gradients(loss, var_list=model.get_thetas())

In [2]:
b, w, h, c = y_train.shape
y = tf.reshape(tf.cast(y_train, tf.int64), (b, w, h))
y = tf.one_hot(y, args.num_classes, on_value=1.0, off_value=0.0)

NameError: name 'y_train' is not defined

In [ ]:
# stem_op = tf.keras.Sequential()
# stem_op.add(tf.keras.layers.Conv2D(
#     3, kernel_size=3, padding='same', use_bias=False))
# stem_op.add(tf.layers.BatchNormalization())

In [ ]:
tf.layers.BatchNormalization()(x_train)

In [ ]:
tf.trainable_variables()

In [ ]:
tf.trainable_variables()

In [ ]:
miou = tf.metrics.mean_iou(
    labels=y,
    predictions=tf.round(logits),
    num_classes=args.num_classes
)

In [ ]:
b = tf.metrics.mean_iou(
    labels=tf.constant([[1, 0, 0], [0, 1, 0]]),
    predictions=tf.constant([[1, 0, 0], [0, 0.99999, 0]]),
    num_classes=2
)

In [21]:
local_op = tf.local_variables_initializer()
global_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run([local_op, global_op])
    out = sess.run(x_train)

OutOfRangeError: End of sequence
	 [[node IteratorGetNext (defined at /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for u'IteratorGetNext':
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 499, in start
    self.io_loop.start()
  File "/usr/local/lib/python2.7/dist-packages/tornado/ioloop.py", line 1073, in start
    handler_func(fd_obj, events)
  File "/usr/local/lib/python2.7/dist-packages/tornado/stack_context.py", line 300, in null_wrapper
    return 